# Stock Prediction with RNN  
RNN을 이용한 간단한 주식 예측 모델을 학습해보겠습니다.

In [ ]:
## library import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

In [ ]:
## 종목코드
stock_code = '105560'

In [ ]:
## HyperParameters
seq_length = 7
data_dim = 5
hidden_size = 10
output_dim = 1
learning_rate = 0.001
training_epochs = 100
batch_size = 32

In [ ]:
!pip install -U finance-datareader

In [ ]:
import FinanceDataReader as fdr

In [ ]:
df = fdr.DataReader(stock_code)

In [ ]:
df

In [ ]:
df.to_csv('stock.csv')

In [ ]:
text = open('stock.csv', 'r').readlines()[1:]
data = [line[:-1].split(',') for line in text]
data = np.array(data)
data = data[:, 1:-1].astype(np.float32)
data[0]

In [ ]:
## 가상화폐 정보 받기
# !pip install -U pyupbit
# import pyupbit
# print(pyupbit.Upbit)
# df = pyupbit.get_ohlcv('KRW-XRP', count=2000)
# df.to_csv('xrp.csv')
# text = open('xrp.csv', 'r').readlines()[1:]
# data = [line[:-1].split(',') for line in text]
# data = np.array(data)
# data = data[:, 1:-1].astype(np.float32)

In [ ]:
## Data Preprocessing
def MinMaxScaler(data):
    ''' Min Max Normalization
    Parameters
    ----------
    data : numpy.ndarray
        input data to be normalized
        shape: [Batch size, dimension]
    Returns
    ----------
    data : numpy.ndarry
        normalized data
        shape: [Batch size, dimension]
    References
    ----------
    .. [1] http://sebastianraschka.com/Articles/2014_about_feature_scaling.html
    '''
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    print(denominator)
    print(np.min(data, 0))
    # noise term prevents the zero division
    result = numerator / (denominator + 1e-7)
    return result, denominator[-2], np.min(data, 0)[-2]

In [ ]:
data, denom, min = MinMaxScaler(data)
x = data
y = data[:, -2:-1]
x.shape, y.shape

In [ ]:
# build a dataset
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    if i % 100 == 0:
      print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

In [ ]:
## Train/Test Split
train_size = int(len(dataY) * 0.7)
train_size = train_size + (batch_size - train_size % batch_size)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

In [ ]:
## Dataset 만들기
train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(
                buffer_size=5000).prefetch(buffer_size=batch_size).batch(batch_size).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((testX, testY)).prefetch(
                buffer_size=batch_size).batch(batch_size)

In [ ]:
## Model 만들기
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(units=hidden_size, return_sequences=True,
                                     input_shape=(trainX.shape[1],trainX.shape[2])))
    model.add(keras.layers.LSTM(units=hidden_size))
    model.add(keras.layers.Dense(units=output_dim))
    return model

In [ ]:
model = create_model()
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate),
              loss='mse',
              metrics=[keras.metrics.RootMeanSquaredError()])
model.summary()

In [ ]:
## Training
model.fit(train_dataset, epochs=training_epochs,
          steps_per_epoch=trainX.shape[0]//batch_size,
          validation_data=test_dataset,
          validation_steps=testX.shape[0]//batch_size+1)

In [ ]:
## 결과확인
prediction = model.predict(test_dataset)
# final_predict * (denom + 1e-7) + min
testY_unscale = testY * (denom + 1e-7) + min
prediction_unscale = prediction * (denom + 1e-7) + min
plt.figure(figsize=(15,7))
plt.plot(testY_unscale)
plt.plot(prediction_unscale)
plt.xlabel("Time Period")
plt.ylabel("Price")
plt.legend(['real', 'prediction'])
plt.show()

In [ ]:
test_input = np.reshape(x[-7:], (-1, 7, 5))
test_input.shape

In [ ]:
final_predict = model.predict(test_input)
final_predict = final_predict * (denom + 1e-7) + min
print(f'다음날 예측 가격: {final_predict}')